<a href="https://colab.research.google.com/github/KartikGarg20526/Chart_Analyzer/blob/main/Finetuning_ChartGemma_ChartQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing Libraries

In [1]:
!pip install transformers peft bitsandbytes datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00


In [2]:
from transformers import (
    AutoProcessor,
    PaliGemmaForConditionalGeneration,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset, Dataset, DatasetDict
from torch.utils.data import Dataset, DataLoader
from typing import List, Dict
from PIL import Image
from io import BytesIO
import torch
import numpy as np
from tqdm import tqdm

##Load Dataset

In [3]:
dataset = load_dataset("ahmed-masry/ChartQA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/219M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/314M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/321M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/50.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/68.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28299 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1920 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['imgname', 'query', 'label', 'type', 'image'],
        num_rows: 28299
    })
    val: Dataset({
        features: ['imgname', 'query', 'label', 'type', 'image'],
        num_rows: 1920
    })
    test: Dataset({
        features: ['imgname', 'query', 'label', 'type', 'image'],
        num_rows: 2500
    })
})

In [5]:
train_ds = dataset['train'].select(range(2000))
val_ds = dataset['val'].select(range(1000))

In [6]:
len(train_ds), len(val_ds)

(2000, 1000)

##Load Processor & Model

In [7]:
processor = AutoProcessor.from_pretrained("ahmed-masry/chartgemma")

preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)
model = PaliGemmaForConditionalGeneration.from_pretrained(
    'ahmed-masry/chartgemma',
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [9]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

##Pytorch Dataset

In [10]:
class ChartGemmaDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx: int) -> Dict:
        sample = self.dataset[idx]
        image = Image.open(BytesIO(sample["image"])).convert('RGB')
        query = sample['query']
        label = sample['label']
        return (image, query, label)
        # return {
        #     "image": Image.open(BytesIO(sample["image"])).convert('RGB'),
        #     "query": sample['query'],
        #     "label": sample['label']
        # }

In [11]:
train_dataset = ChartGemmaDataset(train_ds)

##Define Collate Function

In [12]:
def train_collate_fn(batch, processor, max_length: int = 128):
    images = []
    queries = []
    labels = []
    for example in batch:
        image, query, label = example
        images.append(image)
        queries.append(query)
        labels.append(label)

    inputs = processor(
        text=queries,
        images=images,
        suffix=labels,
        return_tensors="pt",
        padding=True,
        truncation="only_second",
        max_length=max_length,
        tokenize_newline_separately=False
    )

    return inputs

##Training Model

In [13]:
def collate_fn(features):
    return train_collate_fn(features, processor)

training_args = TrainingArguments(
    output_dir="./chartgemma-finetuned",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    save_strategy="epoch",
    learning_rate=1e-4,
    fp16=True,
    warmup_steps=50,
    optim="adamw_torch",
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [14]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


Step,Training Loss
500,0.853000
1000,0.752100
1500,0.707500
2000,0.650800
2500,0.386600
3000,0.426900


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


Step,Training Loss
500,0.853000
1000,0.752100
1500,0.707500
2000,0.650800
2500,0.386600
3000,0.426900
3500,0.370100
4000,0.414700


/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


TrainOutput(global_step=4000, training_loss=0.5701945075988769, metrics={'train_runtime': 8123.9333, 'train_samples_per_second': 0.492, 'train_steps_per_second': 0.492, 'total_flos': 6.028988897279251e+16, 'train_loss': 0.5701945075988769, 'epoch': 2.0})

##Saving Model

In [15]:
trainer.save_model("./final-chartgemma-model")
processor.save_pretrained("./final-chartgemma-model")

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


[]

##Evaluation of model

In [16]:
def compute_metric(gt, pred):
    try:
        gt = float(gt)
        pred = float(pred)
        return abs(gt - pred) / abs(gt) <= 0.05
    except:
        return str(gt).lower() == str(pred).lower()

In [17]:
model_path = "./final-chartgemma-model"
processor = AutoProcessor.from_pretrained(model_path)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load the quantized model
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
model.eval()

correct = 0
total = 0
results = []

for example in tqdm(val_ds):
    try:
        image = Image.open(BytesIO(example["image"])).convert('RGB')
        query = example['query']
        ground_truth = example['label']

        model_inputs = processor(text=query, images=image, return_tensors="pt").to("cuda")

        input_len = model_inputs["input_ids"].shape[1]

        with torch.inference_mode():
            generation = model.generate(**model_inputs, max_new_tokens=128)
            generation = generation[0][input_len:]
            predicted_answer = processor.decode(generation, skip_special_tokens=True)

        is_correct = compute_metric(ground_truth, predicted_answer)
        correct += int(is_correct)
        total += 1

        results.append({
            'query': query,
            'ground_truth': ground_truth,
            'prediction': predicted_answer,
            'is_correct': is_correct
        })

    except Exception as e:
        print(f"Error processing example: {e}")
        continue


final_accuracy = (correct / total) * 100

print("\nEvaluation Results:")
print(f"Total examples evaluated: {total}")
print(f"Correct predictions: {correct}")
print(f"Final accuracy: {final_accuracy:.2f}%")
print(f"Results : {results}")

100%|██████████| 1000/1000 [13:04<00:00,  1.28it/s]


Evaluation Results:
Total examples evaluated: 1000
Correct predictions: 545
Final accuracy: 54.50%
Results : [{'query': "What's the color of graph with 56 as the highest value?", 'ground_truth': 'Blue', 'prediction': 'Green', 'is_correct': False}, {'query': 'In which year the difference between blue and green graph 1?', 'ground_truth': '2018', 'prediction': '2006', 'is_correct': True}, {'query': 'What does the blue line represent?', 'ground_truth': 'Not too much/not at all', 'prediction': 'Not too much/not at all', 'is_correct': True}, {'query': 'What is the max value of blue line?', 'ground_truth': '0.72', 'prediction': '72', 'is_correct': False}, {'query': "What's the percentage of respondents who say Job is a top priority for the president and Congress in 2016?", 'ground_truth': '68', 'prediction': '68', 'is_correct': True}, {'query': 'Which line has the lowest value of 71%?', 'ground_truth': 'Economy', 'prediction': 'Economy', 'is_correct': True}, {'query': 'What is the unfavourab